In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
# os - for saving and loading files
# json - to work with json files
# math - to round up results
# time - to add a short pause to not overwhelm the server
import os, json, math, time

# to make yelpapi calls
from yelpapi import YelpAPI

# progress bar from tqdm_notebook
from tqdm.notebook import tqdm_notebook

In [2]:
# Load the two CSV files
superhero_powers = pd.read_csv(r'C:\Users\lidiv\core\Applying Advanced Transformations (Core)\AdvancedTransformations\superhero_powers - superhero_powers.csv')
superhero_info = pd.read_csv(r'C:\Users\lidiv\core\Applying Advanced Transformations (Core)\AdvancedTransformations\superhero_info - superhero_info.csv')

In [3]:
superhero_info.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


In [4]:
print("Columns in superhero_info DataFrame:")
print(superhero_info.columns)

print("\nColumns in superhero_powers DataFrame:")
print(superhero_powers.columns)


Columns in superhero_info DataFrame:
Index(['Hero|Publisher', 'Gender', 'Race', 'Alignment', 'Hair color',
       'Eye color', 'Skin color', 'Measurements'],
      dtype='object')

Columns in superhero_powers DataFrame:
Index(['hero_names', 'Powers'], dtype='object')


In [5]:
# Add a new column 'Hero' in superhero_info by splitting 'Hero|Publisher'
superhero_info['hero'] = superhero_info['Hero|Publisher'].str.split('|').str[0]

In [6]:
# Add a new column 'Publisher' in superhero_info by splitting 'Hero|Publisher'
superhero_info['Publisher'] = superhero_info['Hero|Publisher'].str.split('|').str[1]

In [7]:
superhero_info.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,hero,Publisher
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",Abin Sur,DC Comics
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",Abomination,Marvel Comics
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}",Absorbing Man,Marvel Comics


In [8]:
# Merge the two DataFrames using the 'hero' column
superheroes = pd.merge(superhero_info, superhero_powers, left_on='hero', right_on='hero_names', how='inner')


In [9]:
# Drop the redundant columns 'Hero|Publisher' and 'hero_names'
superheroes.drop(columns=['Hero|Publisher', 'hero_names'], inplace=True)


In [10]:
superheroes.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,hero,Publisher,Powers
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics,"Accelerated Healing,Durability,Longevity,Super..."
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics,"Agility,Accelerated Healing,Cold Resistance,Du..."
2,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",Abin Sur,DC Comics,Lantern Power Ring
3,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",Abomination,Marvel Comics,"Accelerated Healing,Intelligence,Super Strengt..."
4,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}",Absorbing Man,Marvel Comics,"Cold Resistance,Durability,Energy Absorption,S..."


In [11]:
## slice out a single test coordinate
test_mea = superheroes.loc[1, 'Measurements']
test_mea

"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"

In [12]:

type(test_mea)

str

In [13]:
## Use json.loads on the test coordinate
json.loads(test_mea)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [14]:
## replace single ' with double " 
test_mea = test_mea.replace("'", '"')
test_mea

'{"Height": "191.0 cm", "Weight": "65.0 kg"}'

In [15]:
## Use json.loads on the test coordinate, again
json.loads(test_mea)

{'Height': '191.0 cm', 'Weight': '65.0 kg'}

In [16]:
# viewing type after using json.loads
type(json.loads(test_mea))

dict

In [17]:
## replace ' with " (entire column)
superheroes['Measurements'] = superheroes['Measurements'].str.replace("'", '"')
## apply json.loads
superheroes['Measurements'] = superheroes['Measurements'].apply(json.loads)

In [18]:
## use .apply pd.Series to convert a dict to columns
superheroes['Measurements'].apply(pd.Series)

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg
...,...,...
458,183.0 cm,83.0 kg
459,165.0 cm,52.0 kg
460,66.0 cm,17.0 kg
461,170.0 cm,57.0 kg


In [19]:
## Concatenate the 2 new columns and drop the original.
superheroes = pd.concat([superheroes, superheroes['Measurements'].apply(pd.Series)], axis = 1)
#drop the 'measurement'colunm
superheroes = superheroes.drop(columns = 'Measurements')
superheroes.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,hero,Publisher,Powers,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,"Accelerated Healing,Durability,Longevity,Super...",203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,"Agility,Accelerated Healing,Cold Resistance,Du...",191.0 cm,65.0 kg


In [20]:
superheroes.dtypes['Weight']

dtype('O')

In [21]:
superheroes.dtypes['Height']

dtype('O')

In [22]:
superheroes['Weight'] = superheroes['Weight'].replace('kg', '')


In [23]:
superheroes['Height'] = superheroes['Height'].replace('cm', '')


In [24]:
# Remove non-numeric characters (e.g., 'kg') and convert to float
superheroes['Weight'] = superheroes['Weight'].str.replace(r'[^\d.]', '', regex=True).astype(float)


In [25]:
# Remove non-numeric characters (e.g., 'kg') and convert to float
superheroes['Height'] = superheroes['Height'].str.replace(r'[^\d.]', '', regex=True).astype(float)



In [26]:
superheroes.dtypes['Weight']

dtype('float64')

In [27]:
superheroes.dtypes['Height']

dtype('float64')

In [28]:
superheroes.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,hero,Publisher,Powers,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,"Accelerated Healing,Durability,Longevity,Super...",203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,"Agility,Accelerated Healing,Cold Resistance,Du...",191.0,65.0


In [29]:
superheroes['Powers'].value_counts()

Durability,Super Strength                                                                                                                                                                                                                                                                       4
Agility,Stealth,Marksmanship,Weapons Master,Stamina                                                                                                                                                                                                                                             4
Intelligence                                                                                                                                                                                                                                                                                    4
Agility,Accelerated Healing,Durability,Stealth,Danger Sense,Marksmanship,Animal Attributes,Super Strength,Stamina,Super Speed,Anim

In [30]:
# Converting transactions column into a one-hot-encoded column
exploded = superheroes.explode('Powers')
exploded[['hero', 'Powers']].head()

,hero,Powers
0,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
1,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
2,Abin Sur,Lantern Power Ring
3,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."
4,Absorbing Man,"Cold Resistance,Durability,Energy Absorption,S..."


In [31]:
# Step 1: Replace NaN values with an empty string
superheroes['Powers'].fillna('', inplace=True)

In [32]:
# Step 1: Split the 'Powers' column into lists of powers
superheroes['Powers'] = superheroes['Powers'].str.split(',')

In [33]:
#Create a set of unique powers
all_powers = set()
for powers_list in superheroes['Powers']:
    all_powers.update(powers_list)

In [34]:
#  Create a DataFrame with one-hot-encoded columns
encoded_powers = pd.DataFrame()
for power in all_powers:
    encoded_powers[power] = superheroes['Powers'].apply(lambda x: 1 if power in x else 0)

C:\Users\lidiv\AppData\Local\Temp\ipykernel_22636\2880677313.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_powers[power] = superheroes['Powers'].apply(lambda x: 1 if power in x else 0)
C:\Users\lidiv\AppData\Local\Temp\ipykernel_22636\2880677313.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_powers[power] = superheroes['Powers'].apply(lambda x: 1 if power in x else 0)
C:\Users\lidiv\AppData\Local\Temp\ipykernel_22636\2880677313.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually 

In [35]:
#  Concatenate the one-hot-encoded DataFrame with the original DataFrame
superheroes = pd.concat([superheroes, encoded_powers], axis=1)

In [36]:
#  Drop the original 'Powers' column
superheroes.drop(columns=['Powers'], inplace=True)

In [37]:
superheroes.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,hero,Publisher,Height,Weight,...,Magic,Telekinesis,Vision - Heat,Energy Absorption,Vision - X-Ray,Size Changing,Levitation,Dimensional Awareness,Insanity,Magic Resistance
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,...,0,0,0,0,0,0,0,0,0,0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,...,0,0,0,0,0,0,0,0,0,0
